In [1]:
import time
import pandas as pd
import numpy as np
import datetime
import os
import gc
from functools import partial
import matplotlib.pyplot as plt
from utils import load_data, get_logger, get_data_path
from clean_session import preprocess_sessions
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [90]:
train_input = pd.read_parquet('./cache/train_inputs.snappy')

In [78]:
train_input.isna().sum()

step                                        0
timestamp_session_duration             203755
timestamp_dwell_time_prior_clickout    203755
session_id_size                             0
nf                                     642672
price_0                                     0
price_1                                  3172
price_2                                  6732
price_3                                 10488
price_4                                 14494
price_5                                 18748
price_6                                 23271
price_7                                 27900
price_8                                 32520
price_9                                 37716
price_10                                43479
price_11                                55654
price_12                                67210
price_13                                78151
price_14                                88577
price_15                                98350
price_16                          

In [41]:
train_input.shape

(259427, 59)

In [79]:
train_input.columns

Index(['step', 'timestamp_session_duration',
       'timestamp_dwell_time_prior_clickout', 'session_id_size', 'nf',
       'price_0', 'price_1', 'price_2', 'price_3', 'price_4', 'price_5',
       'price_6', 'price_7', 'price_8', 'price_9', 'price_10', 'price_11',
       'price_12', 'price_13', 'price_14', 'price_15', 'price_16', 'price_17',
       'price_18', 'price_19', 'price_20', 'price_21', 'price_22', 'price_23',
       'price_24', 'n_imps', 'prev_cos_0', 'prev_cos_1', 'prev_cos_2',
       'prev_cos_3', 'prev_cos_4', 'prev_cos_5', 'prev_cos_6', 'prev_cos_7',
       'prev_cos_8', 'prev_cos_9', 'prev_cos_10', 'prev_cos_11', 'prev_cos_12',
       'prev_cos_13', 'prev_cos_14', 'prev_cos_15', 'prev_cos_16',
       'prev_cos_17', 'prev_cos_18', 'prev_cos_19', 'prev_cos_20',
       'prev_cos_21', 'prev_cos_22', 'prev_cos_23', 'prev_cos_24', 'target',
       'pos', 'at'],
      dtype='object')

In [43]:
# 61937/train_input.shape[0]

In [6]:
train_input[train_input.price_0.isna()].head()

,step,timestamp_session_duration,timestamp_dwell_time_prior_clickout,session_id_size,nf,price_0,price_1,price_2,price_3,price_4,...,prev_cos_18,prev_cos_19,prev_cos_20,prev_cos_21,prev_cos_22,prev_cos_23,prev_cos_24,target,pos,at
1,29,263.0,4.828314,2.397895,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0.56,0.0
7,3,50.0,2.890372,1.098612,1.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,6.0
9,1,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,5,NaN,NaN
10,3,157.0,3.951244,1.098612,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0.08,0.0
12,3,37.0,2.302585,1.098612,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.04,7.0


In [9]:
train_sids = np.load('./cache/train_session_ids.npy')

In [8]:
train_raw = load_data('train', nrows=5000000)

[05-18 11:46:57 - utils-106 - load_data - INFO] Loading train using 5,000,000 rows (4,999,976 trimmed) which is 31.38% out of total train data


In [12]:
train_sids[1]

'00011005327e1'

In [13]:
train_raw[train_raw.session_id=='00011005327e1']

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
4300710,M7K6U5Q4ZNBM,00011005327e1,1541539680,1,interaction item image,1216256,US,"Port Richey, USA",tablet,NaN,NaN,NaN
4300711,M7K6U5Q4ZNBM,00011005327e1,1541539680,2,interaction item image,1216256,US,"Port Richey, USA",tablet,NaN,NaN,NaN
4300712,M7K6U5Q4ZNBM,00011005327e1,1541539690,3,interaction item image,1216256,US,"Port Richey, USA",tablet,NaN,NaN,NaN
4300713,M7K6U5Q4ZNBM,00011005327e1,1541539690,4,interaction item image,1216256,US,"Port Richey, USA",tablet,NaN,NaN,NaN
4300714,M7K6U5Q4ZNBM,00011005327e1,1541539690,5,interaction item image,1216256,US,"Port Richey, USA",tablet,NaN,NaN,NaN
4300715,M7K6U5Q4ZNBM,00011005327e1,1541539690,6,interaction item image,1216256,US,"Port Richey, USA",tablet,NaN,NaN,NaN
4300716,M7K6U5Q4ZNBM,00011005327e1,1541539690,7,interaction item image,1216256,US,"Port Richey, USA",tablet,NaN,NaN,NaN
4300717,M7K6U5Q4ZNBM,00011005327e1,1541539690,8,interaction item image,1216256,US,"Port Richey, USA",tablet,NaN,NaN,NaN
4300718,M7K6U5Q4ZNBM,00011005327e1,1541539734,9,interaction item image,57109,US,"Port Richey, USA",tablet,NaN,NaN,NaN
4300719,M7K6U5Q4ZNBM,00011005327e1,1541539734,10,interaction item image,57109,US,"Port Richey, USA",tablet,NaN,NaN,NaN


In [16]:
np.log1p((1541539943-1541539819))

4.8283137373023015

In [25]:
temp = train_raw[train_raw.session_id=='00011005327e1'].prices.str.split('|').iloc[-1]
temp

['68',
 '51',
 '84',
 '84',
 '45',
 '44',
 '171',
 '39',
 '48',
 '118',
 '59',
 '98',
 '57',
 '57',
 '122',
 '100']

In [26]:
temp = [float(i) for i in temp]

In [27]:
temp

[68.0,
 51.0,
 84.0,
 84.0,
 45.0,
 44.0,
 171.0,
 39.0,
 48.0,
 118.0,
 59.0,
 98.0,
 57.0,
 57.0,
 122.0,
 100.0]

In [31]:
np.pad(temp, (0, 25-len(temp)), mode='constant', constant_values=np.nan)

array([ 68.,  51.,  84.,  84.,  45.,  44., 171.,  39.,  48., 118.,  59.,
        98.,  57.,  57., 122., 100.,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan])

In [46]:
df = train_raw[train_raw.action_type=='clickout item'].reset_index(drop=True)

In [47]:
df['prices'] = df['prices'].str.split('|')
df['prices'] = df['prices'].apply(lambda x: [float(p) for p in x])
df['time_steps'] = df['prices'].str.len()
padding_mask = df['time_steps'] < 25
df.drop('time_steps', axis=1, inplace=True)
df.loc[padding_mask, 'prices'] = df.loc[padding_mask, 'prices'].apply(lambda x: np.pad(x, (0, 25-len(x)),
                                                                                   mode='constant',
                                                                                   constant_values=np.nan))

In [56]:
np.array([0,1,10, np.nan]).max()

nan

In [57]:
np.nanmax(np.array([0,1,10, np.nan]))

10.0

In [48]:
df[df.session_id=='00011005327e1'].prices.iloc[-1]

array([ 68.,  51.,  84.,  84.,  45.,  44., 171.,  39.,  48., 118.,  59.,
        98.,  57.,  57., 122., 100.,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan])

In [59]:
def normalize(ps):
    p_arr = np.array(ps)
#     if pd.isna(p_arr.max()):
#         print(p_arr)
    return p_arr / np.nanmax(p_arr)
df['prices_percentage'] = df['prices'].apply(normalize)

In [63]:
68/171

0.39766081871345027

In [61]:
df[df.session_id=='00011005327e1'].prices.iloc[-1]

array([ 68.,  51.,  84.,  84.,  45.,  44., 171.,  39.,  48., 118.,  59.,
        98.,  57.,  57., 122., 100.,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan])

In [60]:
df[df.session_id=='00011005327e1'].prices_percentage.iloc[-1]

array([0.39766082, 0.29824561, 0.49122807, 0.49122807, 0.26315789,
       0.25730994, 1.        , 0.22807018, 0.28070175, 0.69005848,
       0.34502924, 0.57309942, 0.33333333, 0.33333333, 0.71345029,
       0.58479532,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan])

In [44]:
df[[f'price_{i}' for i in range(25)]] = pd.DataFrame(df['prices_percentage'].values.tolist(), index=df.index)

In [45]:
df[df.session_id=='00011005327e1'].iloc[-1]

user_id                                                   M7K6U5Q4ZNBM
session_id                                               00011005327e1
timestamp                                                   1541539943
step                                                                29
action_type                                              clickout item
reference                                                      1333884
platform                                                            US
city                                                  Port Richey, USA
device                                                          tablet
current_filters                                                    NaN
impressions          68607|981941|1216256|1333884|73042|6136340|103...
prices               [68.0, 51.0, 84.0, 84.0, 45.0, 44.0, 171.0, 39...
prices_percentage    [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...
price_0                                                            NaN
price_

In [ ]:
train_input.price_24.hist()

In [ ]:
train_input[[f'price_{i}' for i in range(25)]].describe()

In [66]:
df['timestamp'] = df['timestamp'].apply(lambda ts: datetime.datetime.utcfromtimestamp(ts))


In [70]:
df.timestamp.diff().dt.total_seconds()

0             NaN
1           942.0
2         25379.0
3         36458.0
4           330.0
5          6848.0
6            38.0
7        -45006.0
8         40147.0
9        -27421.0
10        29307.0
11           94.0
12       -30935.0
13       -39027.0
14         5926.0
15        52711.0
16       -48886.0
17        31371.0
18          257.0
19        18584.0
20       -52790.0
21         2281.0
22          143.0
23           30.0
24           20.0
25          145.0
26          102.0
27        58891.0
28       -27088.0
29         1482.0
           ...   
496413   -44112.0
496414      162.0
496415      830.0
496416      161.0
496417      353.0
496418       99.0
496419       87.0
496420       69.0
496421      197.0
496422    27678.0
496423      555.0
496424      381.0
496425   -41223.0
496426    61386.0
496427       28.0
496428      946.0
496429    -9537.0
496430       88.0
496431       38.0
496432   -40843.0
496433        3.0
496434       90.0
496435    -6799.0
496436   -19079.0
496437    

In [72]:
df.timestamp.head()

0   2018-11-01 01:59:03
1   2018-11-01 02:14:45
2   2018-11-01 09:17:44
3   2018-11-01 19:25:22
4   2018-11-01 19:30:52
Name: timestamp, dtype: datetime64[ns]

In [86]:
# test_input
test_input = pd.read_parquet('./cache/test_inputs.snappy')

In [87]:
test_input.shape

(253573, 58)

In [88]:
test_input.columns

Index(['step', 'timestamp_session_duration',
       'timestamp_dwell_time_prior_clickout', 'session_id_size', 'nf',
       'price_0', 'price_1', 'price_2', 'price_3', 'price_4', 'price_5',
       'price_6', 'price_7', 'price_8', 'price_9', 'price_10', 'price_11',
       'price_12', 'price_13', 'price_14', 'price_15', 'price_16', 'price_17',
       'price_18', 'price_19', 'price_20', 'price_21', 'price_22', 'price_23',
       'price_24', 'n_imps', 'prev_cos_0', 'prev_cos_1', 'prev_cos_2',
       'prev_cos_3', 'prev_cos_4', 'prev_cos_5', 'prev_cos_6', 'prev_cos_7',
       'prev_cos_8', 'prev_cos_9', 'prev_cos_10', 'prev_cos_11', 'prev_cos_12',
       'prev_cos_13', 'prev_cos_14', 'prev_cos_15', 'prev_cos_16',
       'prev_cos_17', 'prev_cos_18', 'prev_cos_19', 'prev_cos_20',
       'prev_cos_21', 'prev_cos_22', 'prev_cos_23', 'prev_cos_24', 'pos',
       'at'],
      dtype='object')

In [89]:
test_input['at'].unique()

array([nan,  1.,  8.,  3.,  2.,  0.,  5.,  6.,  4.,  7.,  9.])

In [91]:
train_input['at'].unique()

array([nan,  0.,  6.,  2.,  8.,  5.,  3.,  1.,  7.,  9.,  4.])

In [92]:
train_input.shape

(826505, 59)

In [93]:
test_input.shape

(253573, 58)

---

In [ ]:
from create_model_inputs import compute_session_fts, prepare_data, flogger

In [ ]:
logger = get_logger('create_model_inputs')
Filepath = get_data_path()


In [ ]:
mode='train'
nrows=1000000
logger.info(f'Prepare {mode} data')
t_init = time.time()
df = prepare_data(mode, convert_action_type=True, nrows=nrows, recompute=False)
logger.info('Compute session features')

In [ ]:
str_prices = np.concatenate(df.prices.dropna().str.split('|').values)
str_prices = [int(sp) for sp in str_prices]
# pd.value_counts(str_prices)
_ = plt.hist(str_prices)

In [ ]:
%%time
df = compute_session_fts(df, mode)

In [ ]:
str_prices = np.concatenate(df.prices.dropna().str.split('|').values)
str_prices = [int(sp) for sp in str_prices]
# pd.value_counts(str_prices)
_ = plt.hist(str_prices)

In [ ]:
logger.info('Only select last click-out from each session')
df = df.groupby('session_id').last().reset_index()
flogger(df, 'df shape after only selecting last click-out row each session')

# # log-transform on session_size feature
# logger.info('Log-transform on session_size feature')
# df['session_id_size'] = np.log(df['session_id_size'])

# # log1p-transform on timestamp_dwell_time_prior_clickout but will cliping upper to 1hr
# logger.info('Also log-transform on timestamp_dwell_time_prior_clickout but will cliping upper to 1hr')
# df['timestamp_dwell_time_prior_clickout'] = np.log1p(df['timestamp_dwell_time_prior_clickout'].clip(upper=60 ** 2))

In [ ]:
str_prices = np.concatenate(df.prices.dropna().str.split('|').values)
str_prices = [int(sp) for sp in str_prices]
# pd.value_counts(str_prices)
_ = plt.hist(str_prices)

In [ ]:
df['prices'] = df['prices'].str.split('|')
df['prices_int'] = df['prices'].apply(lambda x: [int(p) for p in x])

In [ ]:
str_prices = np.concatenate(df.prices_int.dropna().values)
# pd.value_counts(str_prices)
_ = plt.hist(str_prices)

In [ ]:
df['time_steps'] = df['prices_int'].str.len()
padding_mask = df['time_steps'] < 25
df.drop('time_steps', axis=1, inplace=True)

In [ ]:
df.prices.head()

In [ ]:
df.prices.iloc[0]

In [ ]:
df[df.prices.str.len()<25].prices.iloc[0]

In [ ]:
x = [int(i) for i in df[df.prices.str.len()<25].prices.iloc[0]]
x

In [ ]:
np.pad(x, (0, 25-len(x)), mode='constant', constant_values=0)

In [ ]:
type(np.nan)

In [ ]:
df.loc[padding_mask, 'prices_int'] = df.loc[padding_mask, 'prices_int'].apply(lambda x: np.pad(x, (0, 25-len(x)),
                                                                                       mode='constant',
                                                                                       constant_values=np.nan))

In [ ]:
str_prices = np.concatenate(df.prices_int.dropna().values)
# pd.value_counts(str_prices)
_ = plt.hist(str_prices)

In [ ]:
def normalize(ps):
    p_arr = np.array(ps)
    return p_arr / (p_arr.max())
df['prices_percentage_int'] = df['prices'].apply(normalize)